# Lesson 2 • Pandas Exercises

Goal: Load/inspect data, filter with masks, group & aggregate, handle missing values, and merge datasets.



In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 10)
np.random.seed(42)


In [3]:
# Utility: create a tiny CSV to read from
import io
csv_text = io.StringIO('''id,age,city,income
1,25,Tel Aviv,5400
2,31,Haifa,7200
3,29,Jerusalem,6600
4,41,Tel Aviv,8800
5,,Haifa,NaN
6,35,Jerusalem,7300
''')


## 1) Series & DataFrame Creation

In [5]:
# 1.1 Create a Series from a Python list of temperatures
temps = [21.5, 22.0, 20.1, 19.8]
# TODO: create `s`
s = pd.Series(data=temps, dtype='float32')
assert isinstance(s, pd.Series) and len(s)==4


In [6]:
# 1.2 Read the CSV above into a DataFrame `df`
df = pd.read_csv(csv_text)
assert list(df.columns) == ['id','age','city','income'] and len(df)==6


In [9]:
# 1.3 Create a DataFrame from dict
data = {
    'name': ['Alice','Bob','Carol','Dan'],
    'score': [88,72,95,60],
    'passed': [True, True, True, False]
}
df2 = pd.DataFrame(data=data)  # Can also be `from_dict(data)` if df already exists
assert {'name','score','passed'} <= set(df2.columns)


## 2) Indexing, Slicing, Filtering

In [6]:
# 2.1 Select rows with age > 30 from `df`
# TODO: set `older`
import io
csv_text = io.StringIO('''id,age,city,income
1,25,Tel Aviv,5400
2,31,Haifa,7200
3,29,Jerusalem,6600
4,41,Tel Aviv,8800
5,,Haifa,NaN
6,35,Jerusalem,7300
''')
df = pd.read_csv(csv_text)
older = df.where(df['age'] > 30)
older.dropna(inplace=True)
assert (older['age'] > 30).all()

older = df.query('age > 30')
assert (older['age'] > 30).all()

older = df.loc[df['age'] > 30]
assert (older['age'] > 30).all()


In [7]:
# 2.2 Select only 'city' and 'income' columns
cols = df[['city', 'income']]
assert list(cols.columns) == ['city','income']


In [8]:
# 2.3 Use .loc to select rows with id in [2,4] and columns ['age','income']
subset = df.loc[df['id'].isin([2,4]), ['age', 'income']]
print(subset)
assert subset.shape == (2,2)


    age  income
1  31.0  7200.0
3  41.0  8800.0


In [9]:
# 2.4 Use .iloc to select first 3 rows and the last 2 columns
iloc_part = df.iloc[0:3, -2:]
assert iloc_part.shape == (3,2)


## 3) Groupby & Aggregation

In [11]:
# 3.1 Group by 'city' compute mean income (ignore NaNs)
df = pd.read_csv(io.StringIO(csv_text.getvalue()))
mean_income_by_city = df.groupby(df['city'],dropna=True)['income'].mean()
assert isinstance(mean_income_by_city, pd.Series)


In [21]:
# 3.2 Group by city, compute count and max age
# agg lets perform several operations on a column in one line in "1 call" (behind the scene it is probably more) 
agg_stats = df.groupby(df['city']).agg({'income': ['count', 'mean'], 'age': ['count', 'mean']})  # use .agg({'age':['count','max']})
print(agg_stats, "\n", df)
assert 'count' in agg_stats.columns.get_level_values(1).tolist()


          income           age      
           count    mean count  mean
city                                
Haifa          1  7200.0     1  31.0
Jerusalem      2  6950.0     2  32.0
Tel Aviv       2  7100.0     2  33.0 
    id   age       city  income
0   1  25.0   Tel Aviv  5400.0
1   2  31.0      Haifa  7200.0
2   3  29.0  Jerusalem  6600.0
3   4  41.0   Tel Aviv  8800.0
4   5   NaN      Haifa     NaN
5   6  35.0  Jerusalem  7300.0


In [ ]:
# 3.3 Create a new column 'income_k' = income / 1000 then group by city and compute mean
df = pd.read_csv(io.StringIO(csv_text.getvalue()))
df['income_k'] = df['income'] / 1000
mean_k = df.groupby(df['city'], dropna=True)['income_k'].mean(numeric_only=True)
print(mean_k)
assert isinstance(mean_k, pd.Series)


city
Haifa        7.20
Jerusalem    6.95
Tel Aviv     7.10
Name: income_k, dtype: float64


## 4) Sorting & Missing Values

In [37]:
# 4.1 Sort df by income descending (NaNs last)
df = pd.read_csv(io.StringIO(csv_text.getvalue()))
sorted_df = df.sort_values(by='income', ascending=False, na_position='last')
assert sorted_df['income'].iloc[0] >= sorted_df['income'].iloc[1]


In [90]:
# 4.2 Count missing values per column
na_counts = df.isna().sum()
print(na_counts)
assert isinstance(na_counts, pd.Series) and 'age' in na_counts.index

id        0
age       1
city      0
income    1
dtype: int64


In [ ]:
# 4.3 Fill missing age with the mean age and drop rows where income is NaN

csv_text = '''age,city,income
25,Tel Aviv,5400
31,Haifa,7200
,Jerusalem,6600
41,Tel Aviv,
,Haifa,
35,Jerusalem,7300
22,,4800
,Berlin,
28,Paris,5900
,Tokyo,'''

df = pd.read_csv(io.StringIO(csv_text))
print(df)
df['age'] = df['age'].fillna(df['age'].mean())
df = df.dropna(subset=['city', 'income'])
print(df)
assert df['age'].isna().sum() == 0 and df['income'].isna().sum() == 0

    age       city  income
0  25.0   Tel Aviv  5400.0
1  31.0      Haifa  7200.0
2   NaN  Jerusalem  6600.0
3  41.0   Tel Aviv     NaN
4   NaN      Haifa     NaN
5  35.0  Jerusalem  7300.0
6  22.0        NaN  4800.0
7   NaN     Berlin     NaN
8  28.0      Paris  5900.0
9   NaN      Tokyo     NaN
         age       city  income
0  25.000000   Tel Aviv  5400.0
1  31.000000      Haifa  7200.0
2  30.333333  Jerusalem  6600.0
5  35.000000  Jerusalem  7300.0
8  28.000000      Paris  5900.0


## 5) Merge / Join (Bonus)

In [106]:
# 5.1 Merge two DataFrames on 'id'
left  = pd.DataFrame({'id':[1,2,3], 'group':['A','B','A']})
right = pd.DataFrame({'id':[2,3,4], 'score':[90,75,60]})
# TODO: set `merged`
merged = pd.merge(left, right, on='id')
print(left)
print(right)
print(merged)
assert set(merged.columns) == {'id','group','score'} and len(merged)==2


   id group
0   1     A
1   2     B
2   3     A
   id  score
0   2     90
1   3     75
2   4     60
   id group  score
0   2     B     90
1   3     A     75
